# Atproto OAuth

This turned out to be a major pain in the butt to get to work with Deno. See
[this](https://github.com/bluesky-social/atproto/discussions/3303)

Start by patching `jwk-jose` from
https://github.com/bluesky-social/atproto/tree/main/packages/oauth/jwk-jose.
Basically copy what's there in atproto repo and use jose from denoland URL.
Also, patch this line to include algorithm explicitly (see
[this](https://github.com/panva/jose/discussions/648)):

```ts
return (this.#keyObj ||= await importJWK(this.jwk as JWK, "ES256"));
```

In [ ]:
// |export

import { JwtVerifyError } from "@atproto/jwk";
import {
  errors,
  exportJWK,
  generateKeyPair,
  type GenerateKeyPairOptions,
  type GenerateKeyPairResult,
  importJWK,
  importPKCS8,
  type JWK,
  jwtVerify,
  type JWTVerifyOptions,
  type KeyLike,
  SignJWT,
} from "https://deno.land/x/jose@v5.9.6/index.ts";

import {
  Jwk,
  JwkError,
  jwkValidator,
  JwtCreateError,
  JwtHeader,
  JwtPayload,
  Key,
  SignedJwt,
  VerifyOptions,
  VerifyPayload,
  VerifyResult,
} from "@atproto/jwk";

export function either<T extends string | number | boolean>(
  a?: T,
  b?: T,
): T | undefined {
  if (a != null && b != null && a !== b) {
    throw new TypeError(`Expected "${b}", got "${a}"`);
  }
  return a ?? b ?? undefined;
}

const { JOSEError } = errors;

export type Importable = string | KeyLike | Jwk;

export type { GenerateKeyPairOptions, GenerateKeyPairResult };

export class JoseKey extends Key {
  #keyObj?: KeyLike | Uint8Array;

  protected async getKey() {
    try {
      return (this.#keyObj ||= await importJWK(this.jwk as JWK, "ES256"));
    } catch (cause) {
      throw new JwkError("Failed to import JWK", undefined, { cause });
    }
  }

  async createJwt(header: JwtHeader, payload: JwtPayload) {
    if (header.kid && header.kid !== this.kid) {
      throw new JwtCreateError(
        `Invalid "kid" (${header.kid}) used to sign with key "${this.kid}"`,
      );
    }

    if (!header.alg || !this.algorithms.includes(header.alg)) {
      throw new JwtCreateError(
        `Invalid "alg" (${header.alg}) used to sign with key "${this.kid}"`,
      );
    }

    const keyObj = await this.getKey();
    return new SignJWT(payload)
      .setProtectedHeader({ ...header, kid: this.kid })
      .sign(keyObj) as Promise<SignedJwt>;
  }

  async verifyJwt<
    P extends VerifyPayload = JwtPayload,
    C extends string = string,
  >(token: SignedJwt, options?: VerifyOptions<C>): Promise<VerifyResult<P, C>> {
    try {
      const keyObj = await this.getKey();
      const result = await jwtVerify(token, keyObj, {
        ...options,
        algorithms: this.algorithms,
      } as JWTVerifyOptions);

      return result as VerifyResult<P, C>;
    } catch (error) {
      if (error instanceof JOSEError) {
        throw new JwtVerifyError(error.message, error.code, { cause: error });
      } else {
        throw JwtVerifyError.from(error);
      }
    }
  }

  static async generateKeyPair(
    allowedAlgos: readonly string[] = ["ES256"],
    options?: GenerateKeyPairOptions,
  ) {
    if (!allowedAlgos.length) {
      throw new JwkError("No algorithms provided for key generation");
    }

    const errors: unknown[] = [];
    for (const alg of allowedAlgos) {
      try {
        return await generateKeyPair(alg, options);
      } catch (err) {
        errors.push(err);
      }
    }

    throw new JwkError("Failed to generate key pair", undefined, {
      cause: new AggregateError(errors, "None of the algorithms worked"),
    });
  }

  static async generate(
    allowedAlgos: string[] = ["ES256"],
    kid?: string,
    options?: Omit<GenerateKeyPairOptions, "extractable">,
  ) {
    const kp = await this.generateKeyPair(allowedAlgos, {
      ...options,
      extractable: true,
    });
    return this.fromImportable(kp.privateKey, kid);
  }

  static async fromImportable(
    input: Importable,
    kid?: string,
  ): Promise<JoseKey> {
    if (typeof input === "string") {
      // PKCS8
      if (input.startsWith("-----")) {
        // The "alg" is only needed in WebCrypto (NodeJS will be fine)
        return await this.fromPKCS8(input, "", kid);
      }

      // Jwk (string)
      if (input.startsWith("{")) {
        return this.fromJWK(input, kid);
      }

      throw new JwkError("Invalid input");
    }

    if (typeof input === "object") {
      // Jwk
      if ("kty" in input || "alg" in input) {
        return this.fromJWK(input, kid);
      }

      // KeyLike
      return this.fromKeyLike(input, kid);
    }

    throw new JwkError("Invalid input");
  }

  /**
   * @see {@link exportJWK}
   */
  static async fromKeyLike(
    keyLike: KeyLike | Uint8Array,
    kid?: string,
    alg?: string,
  ): Promise<JoseKey> {
    const jwk = await exportJWK(keyLike);
    if (alg) {
      if (!jwk.alg) jwk.alg = alg;
      else if (jwk.alg !== alg) throw new JwkError('Invalid "alg" in JWK');
    }
    // @ts-ignore cannot convert JWK to string
    return this.fromJWK(jwk, kid);
  }

  /**
   * @see {@link importPKCS8}
   */
  static async fromPKCS8(
    pem: string,
    alg: string,
    kid?: string,
  ): Promise<JoseKey> {
    const keyLike = await importPKCS8(pem, alg, { extractable: true });
    return this.fromKeyLike(keyLike, kid);
  }

  static fromJWK(
    input: string | Record<string, unknown>,
    inputKid?: string,
  ): JoseKey {
    const jwk = typeof input === "string" ? JSON.parse(input) : input;
    if (!jwk || typeof jwk !== "object") throw new JwkError("Invalid JWK");

    const kid = either(jwk.kid, inputKid);
    const use = jwk.use || "sig";

    return new JoseKey(jwkValidator.parse({ ...jwk, kid, use }));
  }
}

In [ ]:
// |export

import { InternalStateData, OAuthClient, Session } from "@atproto/oauth-client";
import { createHash, randomBytes } from "node:crypto";

// set this to the public URL of the app
const publicUrl = "https://tinychat.ngrok.app";

// in memory store for state and session data
const stateStore: Map<string, InternalStateData> = new Map();
const sessionStore: Map<string, Session> = new Map();

export const getOAuthClient = () =>
  new OAuthClient({
    handleResolver: "https://api.bsky.app", // backend instances should use a DNS based resolver
    responseMode: "query",
    clientMetadata: {
      client_name: "AT Protocol Express App",
      client_id: publicUrl + "/client-metadata.json",
      client_uri: publicUrl,
      redirect_uris: [`${publicUrl}/oauth/callback`],
      scope: "atproto transition:generic",
      grant_types: ["authorization_code", "refresh_token"],
      response_types: ["code"],
      application_type: "web",
      token_endpoint_auth_method: "none",
      dpop_bound_access_tokens: true,
    },

    stateStore: {
      // A store for saving state data while the user is being redirected to the
      // authorization server.

      set(key: string, internalState: InternalStateData) {
        stateStore.set(key, internalState);
      },
      get(key: string): InternalStateData | undefined {
        return stateStore.get(key);
      },
      del(key: string) {
        stateStore.delete(key);
      },
    },

    sessionStore: {
      // A store for saving session data.

      set(sub: string, session: Session) {
        sessionStore.set(sub, session);
      },
      get(sub: string): Session | undefined {
        return sessionStore.get(sub);
      },
      del(sub: string) {
        stateStore.delete(sub);
      },
    },

    runtimeImplementation: {
      // A runtime specific implementation of the crypto operations needed by the
      // OAuth client. See "@atproto/oauth-client-browser" for a browser specific
      // implementation. The following example is suitable for use in NodeJS.

      createKey(algs: string[]) {
        return JoseKey.generate(algs);
      },
      getRandomValues: randomBytes,
      digest(
        bytes: Uint8Array,
        algorithm: { name: string },
      ) {
        return createHash(algorithm.name).update(bytes).digest();
      },
    },
  });

Let's set up test server

In [ ]:
import { Hono } from "hono";
import { logger } from "hono/logger";

const app = new Hono();
const oauthClient = getOAuthClient();

app.use("*", logger());
app.get("/client-metadata.json", async (c) => {
  return c.json(oauthClient.clientMetadata);
});

app.get("/oauth/callback", async (c) => {
  const params = new URLSearchParams(c.req.url.split("?")[1]);

  const { session } = await oauthClient.callback(params);

  console.log("session", session);

  const oauthSession = oauthClient.restore(session.did);

  console.log("oauth session", oauthSession);

  return c.redirect("/");
});

app.get("/login", async (c) => {
  const url = await oauthClient.authorize("callmephilip.com", {
    scope: "atproto transition:generic",
  });
  return c.redirect(url.toString());
});

// run locally to test
// Deno.serve(app.fetch);

Hono {
  get: [Function (anonymous)],
  post: [Function (anonymous)],
  put: [Function (anonymous)],
  delete: [Function (anonymous)],
  options: [Function (anonymous)],
  patch: [Function (anonymous)],
  all: [Function (anonymous)],
  on: [Function (anonymous)],
  use: [Function (anonymous)],
  router: SmartRouter { name: "SmartRouter" },
  getPath: [Function: getPath],
  _basePath: "/",
  routes: [
    { path: "/*", method: "ALL", handler: [AsyncFunction: logger] },
    {
      path: "/client-metadata.json",
      method: "GET",
      handler: [AsyncFunction (anonymous)]
    },
    {
      path: "/oauth/callback",
      method: "GET",
      handler: [AsyncFunction (anonymous)]
    },
    {
      path: "/login",
      method: "GET",
      handler: [AsyncFunction (anonymous)]
    }
  ],
  errorHandler: [Function: errorHandler],
  onError: [Function: onError],
  notFound: [Function: notFound],
  fetch: [Function: fetch],
  request: [Function: request],
  fire: [Function: fire]
}